In [4]:
using COVIDIncidencePOMDPs
using POMCPOW
using POMDPModelTools
using Distributions
using POMDPSimulators
using POMDPs
using POMDPPolicies
using Random
using ParticleFilters
using POMCPOW

In [3]:
p = PARAM_MAPPING["good"]

k, lam = get_weibull_params(p.MEAN_GEN_TIME, p.STD_GEN_TIME)
gen_func = discrete_weibull(k, lam, p.TOTAL_TIME)
base_R0 = calc_R0(k, lam, p.EXP_GROWTH, p.TOTAL_TIME)
initial_incidences = ones(Float64, p.TOTAL_TIME) * p.INFECTION_SEED
#initial_belief = InitialBeliefDistribution(initial_incidences, p.NUM_LEVELS)
initial_belief = InitialBeliefDistribution(initial_incidences, 10.0, p.NUM_LEVELS)

covid_pomdp = CovidPOMDP(base_R0, gen_func, initial_incidences, p)

solver = POMCPOWSolver(max_depth=20, tree_queries=1,
    k_observation=6.0, alpha_observation=1/20., enable_action_pw=false)#,
    #estimate_value=RolloutEstimator(FixedSolver()))

planner = solve(solver, covid_pomdp)
updater = SIRParticleFilter(covid_pomdp, 100)

#stepper = stepthrough(covid_pomdp, planner, updator, initial_belief, "b,s,sp,a,o,r", max_steps=200)

sim = HistoryRecorder(max_steps=100)
stepper = simulate(sim, covid_pomdp, planner, updater, initial_belief)

for (s, b, a, r, sp, o) in stepper
    #@show s, b, a, r, sp, o
    @show r
end

Noising mobilities | std: [0.159, 0.149, 0.185, 0.124, 0.164]
Noising confounds | std: [6.82524674412e9, 1.77005605606039e12, 1.5558643518773e10, 1.02412395113e8, 6.89008819020368e11]
Noising mobilities | std: [0.258, 0.086, 0.07, 0.034, 0.295]
Noising confounds | std: [0.0, 0.866, 0.0, 0.866, 0.0]
Noising mobilities | std: [0.036, 0.013, 0.033, 0.012, 0.029]
Noising confounds | std: [0.007, 0.387, 0.005, 0.413, 0.005]
Noising mobilities | std: [0.034, 0.012, 0.038, 0.011, 0.032]
Noising confounds | std: [0.01, 0.279, 0.007, 0.247, 0.007]
Noising mobilities | std: [0.028, 0.004, 0.016, 0.004, 0.02]
Noising confounds | std: [0.009, 0.077, 0.003, 0.064, 0.005]
Noising mobilities | std: [0.028, 0.006, 0.02, 0.005, 0.02]
Noising confounds | std: [0.011, 0.071, 0.007, 0.06, 0.009]
Noising mobilities | std: [0.032, 0.006, 0.023, 0.006, 0.022]
Noising confounds | std: [0.014, 0.065, 0.011, 0.061, 0.011]
Noising mobilities | std: [0.027, 0.007, 0.022, 0.008, 0.018]
Noising confounds | std: [0.

Noising mobilities | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising confounds | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising mobilities | std: [0.028, 0.024, 0.022, 0.006, 0.028]
Noising confounds | std: [0.011, 0.011, 0.009, 0.01, 0.01]
Noising mobilities | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising confounds | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising mobilities | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising confounds | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising mobilities | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising confounds | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising mobilities | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising confounds | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising mobilities | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising confounds | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising mobilities | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising confounds | std: [0.1, 0.1, 0.1, 0.1, 0.1]
Noising mobilities | std: [0.027, 0.019, 0.022, 0.005, 0.023]
Noising confounds | std: [0.011, 0.009, 0.01, 0.01, 0.01]
Noising mobilities | std: [0.1, 0.1, 0